# Многонишково програмиране с Python

План на лекцията:
- [ ] Скалиране от 1 към много
- [ ] Какво е нишка ?
- [ ] Работа с нишки (Threads)
- [ ] Python GIL
- [ ] Threads, round 2
- [ ] "Многопроцесност"
- [ ] Бъдещето
- [ ] asyncio


## Какво е нишка ?
- Какво е нишка принципно ?
- Зелени нишки

## Работа с нишки (Threads)
https://docs.python.org/3.10/library/threading.html?highlight=threading#module-threading
- Стартиране на нишка
- Изчакване на нишка
- Разглеждане на performance-а

## Python GIL
https://docs.python.org/3.10/glossary.html#term-global-interpreter-lock
- Какво е GIL ?
- Как работи

## Threads, round 2
https://docs.python.org/3.10/library/threading.html?highlight=threading#module-threading
- Семафори
- Lock
- Condition
- Event

## "Многопроцесност"
https://docs.python.org/3.10/library/multiprocessing.html#module-multiprocessing

## Бъдещето
https://docs.python.org/3.10/library/concurrent.futures.html#module-concurrent.futures

## asyncio
https://docs.python.org/3.10/library/asyncio.html#module-asyncio

## Скалиране от 1 към много

Нека се върнем към едни по-прости времена - когато процесорът (да го наречем "v1") може да изпълнява една-единствена задача. Нека освен този прост процесор, да имаме и набор от задачи - `[t1, t2, t3]`.
Процесорът взима първата задача, изпълнява я, взема втората задача, изпълнява нея, и така докато не мине през всички задачи.

(Картинка, на която t1 -> t2 ---> t3 -->)

Нека си представим, че задачата `t2` отнема повече време, защото трябва да изчака за някакъв ресурс (например файлова операция). В това време, нашия процесор v1 чака, и не прави никакви изчисления. Това със сигурност не е оптимално.

### Какво е конкурентност ?

Нека да вземем един нов, по-добър процесор ("v2"), който може да разпознае кога даден процес чака за ресурс, и в такава ситуация да даде приоритет на друг процес. 
В тази ситуация, когато `t2` чака за ресурс, процесорът може да пусне `t3` по-рано. Когато ресурсът на `t2` стане наличен, процесорът ще спре `t3`, и ще даде време обратно на `t2`.   

(Картинка, на която t1 -> t2 -> t3 --> t2 ->)

Това изпълнение, ни позволява да изпълним задачите си по-бързо - вместо да чакаме `t2` да приключи, това време в чакане може да се използва за изпълнението на `t3`. 

Този модел на изпълнение, в който изпълнението на следващата задача не е нужно да изчаква приключването на текущата, се нарича "конкурентно изпълнение".
Важно е да се отбележи, че конкурентното изпълнение не е едновременно (паралелно) изпълнение на задачи - продължаваме да изпълняваме по една задача в даден момент, просто редът им не е детерминиран. Ние не знаем в какъв ред ще се изпълнят задачите и коя от тях ще бъде изпълнена първа.

### Какво е паралелизъм ?

Колкото и оптимално да е раздаването на процесорно време от нашия v2 процесор, той все още не може да прави паралелни изчисления. Затова нека въведем нашия "v3" процесор - той ще е два v2 процесора "залепени" един за друг. Това би означавало, че нашия v3 процесор може да изпълнява две задачи едновременно (паралелно).  

### Модерните процесори

Модерните процесори са изградени от няколко ядра (8, 10, 16), които могат да изпълняват различни задачи паралелно. В идеалния свят, ако нашата задача отнема `t` време на 1 ядро, то на `n` ядра, би трябвало да отнеме `t/n` време.